### chap2

In [1]:
import spacy
import pandas as pd
import re

In [2]:
# ginza model
nlp=spacy.load('ja_ginza')

In [9]:
# data path
D_path='./corpus2/'
topics=['Gourmet', 'Keitai', 'Kyoto', 'Sports']

In [10]:
cols=['doc', 'label']
df=pd.DataFrame(index=[], columns=cols)
regex=re.compile(r'(［.*］)(.*)')

for topic in topics:
    data = pd.read_table(D_path+topic+'.tsv', header=None, dtype=str, usecols=[1])
    for d in data.itertuples():
        m=regex.match(d[1])
        # first line of the docment
        if m!=None:
            if d[0]!=0:
                df=df.append(pd.DataFrame([doc.strip(), topic], index=cols).T)
            sent=m.groups()[1]
            doc=''
        else:
            sent=d[1]
        doc=doc+' '+' '.join([t.text for t in nlp(sent)])
    df=df.append(pd.DataFrame([doc.strip(), topic], index=cols).T)

In [11]:
df

,doc,label
0,烏丸 六角 の おかき 屋 さん 六角堂 の 前 に ある 、 蕪村 庵 と いう お店 に...,Gourmet
0,河原町 の 居酒屋 この 間 先輩 たち に つれ られ 、 河原町 の 居酒屋 へ 行っ ...,Gourmet
0,ちょっと 贅沢 ほんの ちょこっと な ん だ けど 、 贅沢 し たい とき に 何 を ...,Gourmet
0,食欲 の 秋 最近 おなか が 減っ て 仕方 あり ませ ん か 。 仕方 あり ませ ん...,Gourmet
0,季節 限定 も のっ て … １０ 月 と いえ ば 秋 本番 で ある 。 栗 や 秋刀魚...,Gourmet
...,...,...
0,クライマックス シリーズ 今年 の 阪神 は なん やっ た ん や ！ ！ 初め 弱く て...,Sports
0,京都 サンガ に つい て この ブログ の 右側 に 検索 窓 ある じゃ ない です か...,Sports
0,サッカー から 伺える ルール と の 付き合い 方 私自身 は 全く の サッカー 素人 ...,Sports
0,する か 見る か スポーツ は する 派 です か ？ し たい 見る 派 です か ？ ...,Sports


In [12]:
df['label'].value_counts(sort=False)

Sports     22
Gourmet    57
Keitai     79
Kyoto      91
Name: label, dtype: int64

### Document Classification

#### Naive Bayes

#### Logistic Regression

#### Online SVM